In [ ]:
import nltk
import nltk.translate.bleu_score as bleu
import nltk.translate.gleu_score as gleu
import math
import numpy as np
import os
try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

!pip install bert-embedding
from bert_embedding import BertEmbedding
bert_E = BertEmbedding()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 13.8MB 266kB/s 
     |████████████████████████████████| 29.6MB 118kB/s 
     |████████████████████████████████| 215kB 54.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-cp36-none-any.whl size=259917 sha256=0c7954f68a1803b9da745a0f6da9810027cf9552ea9956e56c0d10915c8107cf
  Stored in directory: /root/.cache/pip/wheels/ff/48/ac/a77c79aa416ba6dd7bf487f2280b0471034f66141617965914
Successfully built gluonnlp
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tifffile 2020.9.3 has requirement numpy>=1.15.1, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR

Vocab file is not found. Downloading.


In [ ]:
!python -m spacy download pt_core_news_sm

import pt_core_news_sm
import nltk
nltk.download('stopwords')
nlp = pt_core_news_sm.load()

⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=f0862570dc6b6d4175ced2854f60c5dfb8a02c3123c3aa737f875720e5b6cb74
  Stored in directory: /tmp/pip-ephem-wheel-cache-eo5a2j86/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def token_list(embeddings, no_sep=False):
    """
    Returns with the tokens of the embedding data from the BertEmbedding.

    Params:
        embeddings: The embedding data from BertEmbedding
        no_sep: If True, the separators are trimmed.
    Return:
        tokens: list of tokens
    """
    if no_sep:
        return embeddings[0][0][1:-1]
    return embeddings[0][0]


def sentence_embs(embeddings):
    """Return with the sentence level embeddings"""
    return embeddings[0][1][0]

def prep(sentence):
    """Return with tokens and sentence level embeddings"""
    embs = bert_E([sentence])
    tokens = token_list(embs, no_sep=True)
    se = sentence_embs(embs)
    return tokens, se


def mapfunct(x, type='exp', n=0.2):
    """
    Map 0-inf to 1-0 with some function
    
    Type:
        inverse: 1/(1+n*x)
        arctan: 1-2/pi*arctan(x)
        exp: (1/(1+n))^x
    """
    if type=='inverse':
        return 1/(1+n*x)
    if type=='arctan':
        return 1-2/math.pi*math.atan(n*x)
    if type=='exp':
        return (1/(1+n))**x
    else:
        raise(NotImplementedError("Function not implemented"))


def square_rooted(x):
    return math.sqrt(sum([a*a for a in x]))


def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return numerator/float(denominator)

In [ ]:
s0 = "James Cook was a very good man and a loving husband."
s1 = "James Cook was a very nice man and a loving husband."
s2 = "James Cook was a bad man and a terrible husband."
s3 = "James Cook was a nice person and a good husband."
s4 = "The sky is blue today and learning history is important."

In [ ]:
r0, e0 = prep(s0)
r1, e1 = prep(s1)
r2, e2 = prep(s2)
r3, e3 = prep(s3)
r4, e4 = prep(s4)

In [ ]:
t0 = nlp(s0)
t1 = nlp(s1)
t2 = nlp(s2)
t3 = nlp(s3)
t4 = nlp(s4)

In [ ]:
SmoothingFunction = nltk.translate.bleu_score.SmoothingFunction()

In [ ]:
print("r0-r0 bleu score: ", bleu.sentence_bleu([r0], r0, smoothing_function=SmoothingFunction.method0))
print("r0-r1 bleu score: ", bleu.sentence_bleu([r1], r0, smoothing_function=SmoothingFunction.method0))
print("r0-r2 bleu score: ", bleu.sentence_bleu([r2], r0, smoothing_function=SmoothingFunction.method0))
print("r0-r3 bleu score: ", bleu.sentence_bleu([r3], r0, smoothing_function=SmoothingFunction.method0))
print("r0-r4 bleu score: ", bleu.sentence_bleu([r4], r0, smoothing_function=SmoothingFunction.method0))
print("")
print("r0-r0 bleu score: ", bleu.sentence_bleu([r0], r0, smoothing_function=SmoothingFunction.method2))
print("r0-r1 bleu score: ", bleu.sentence_bleu([r1], r0, smoothing_function=SmoothingFunction.method2))
print("r0-r2 bleu score: ", bleu.sentence_bleu([r2], r0, smoothing_function=SmoothingFunction.method2))
print("r0-r3 bleu score: ", bleu.sentence_bleu([r3], r0, smoothing_function=SmoothingFunction.method2))
print("r0-r4 bleu score: ", bleu.sentence_bleu([r4], r0, smoothing_function=SmoothingFunction.method2))
print("")
print("r0-r0 gleu score: ", gleu.sentence_gleu([r0], r0))
print("r0-r1 gleu score: ", gleu.sentence_gleu([r1], r0))
print("r0-r2 gleu score: ", gleu.sentence_gleu([r2], r0))
print("r0-r3 gleu score: ", gleu.sentence_gleu([r3], r0))
print("r0-r4 gleu score: ", gleu.sentence_gleu([r4], r0))
print("")
print("e0-e0 Euclid distance:", np.linalg.norm(e0-e0))
print("e0-e1 Euclid distance:", np.linalg.norm(e1-e0))
print("e0-e2 Euclid distance:", np.linalg.norm(e2-e0))
print("e0-e3 Euclid distance:", np.linalg.norm(e3-e0))
print("e0-e4 Euclid distance:", np.linalg.norm(e4-e0))
print("")
print("e0-e0 Euclid distance:", str(mapfunct(np.linalg.norm(e0-e0))))
print("e0-e1 Euclid distance:", str(mapfunct(np.linalg.norm(e1-e0))))
print("e0-e2 Euclid distance:", str(mapfunct(np.linalg.norm(e2-e0))))
print("e0-e3 Euclid distance:", str(mapfunct(np.linalg.norm(e3-e0))))
print("e0-e4 Euclid distance:", str(mapfunct(np.linalg.norm(e4-e0))))
print("")
print("e0-e0 cosine-similarity:", cosine_similarity(e0,e0))
print("e0-e1 cosine-similarity:", cosine_similarity(e1,e0))
print("e0-e2 cosine-similarity:", cosine_similarity(e2,e0))
print("e0-e3 cosine-similarity:", cosine_similarity(e3,e0))
print("e0-e4 cosine-similarity:", cosine_similarity(e4,e0))
print("")
print("t0-t0 spacy similarity", t0.similarity(t0))
print("t0-t1 spacy similarity", t1.similarity(t0))
print("t0-t2 spacy similarity", t2.similarity(t0))
print("t0-t3 spacy similarity", t3.similarity(t0))
print("t0-t4 spacy similarity", t4.similarity(t0))

r0-r0 bleu score:  1.0
r0-r1 bleu score:  0.6580370064762462
r0-r2 bleu score:  0.5280972216470737
r0-r3 bleu score:  0.4132584091896901
r0-r4 bleu score:  0.5623413251903491

r0-r0 bleu score:  1.0
r0-r1 bleu score:  0.7016879391277372
r0-r2 bleu score:  0.3508439695638686
r0-r3 bleu score:  0.2998221389342337
r0-r4 bleu score:  0.12605968092174913

r0-r0 gleu score:  1.0
r0-r1 gleu score:  0.7058823529411765
r0-r2 gleu score:  0.38235294117647056
r0-r3 gleu score:  0.3235294117647059
r0-r4 gleu score:  0.029411764705882353

e0-e0 Euclid distance: 0.0
e0-e1 Euclid distance: 1.9738714
e0-e2 Euclid distance: 3.6317627
e0-e3 Euclid distance: 3.0969253
e0-e4 Euclid distance: 17.017267

e0-e0 Euclid distance: 1.0
e0-e1 Euclid distance: 0.6977605424333337
e0-e2 Euclid distance: 0.5157420006874345
e0-e3 Euclid distance: 0.568566934219589
e0-e4 Euclid distance: 0.04493156820402338

e0-e0 cosine-similarity: 1.0
e0-e1 cosine-similarity: 0.9900622593588156
e0-e2 cosine-similarity: 0.965961241983

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: Model